In [1]:
! pip install transformers
! pip install datasets

In [2]:
cd '/content/drive/MyDrive/CAMemBERT2'

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer_f import MultiTaskModelTrainer
from trainer_utils import (training_args,training_kwargs,get_dataset,
                           freeze_layers,normalise_scores,create_params_grid)

In [6]:
for pretrained_model_name in ['distilroberta-base','bert-base-uncased']:
    model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
    model_n_layers = 6 if 'roberta' in pretrained_model_name else 12
    params_grid = sorted(create_params_grid({'tasks':[['aes'],['ged'],['ner']],
                                      'lr':[1e-5,4e-5,8e-5],
                                      'frozen_layers':[None,model_n_layers/2,model_n_layers]}
                                      )
                        , key=lambda k: k['tasks'])

    prev_task = None
    for i,params in enumerate(params_grid):
        print(params)
        if params['tasks']!=prev_task:
            dataset_obj,dataset_dict = get_dataset(params['tasks'],pretrained_model=pretrained_model_name,max_length=512)
            dataset_dict = dataset_dict.map(normalise_scores)

        trainer_kwargs,model_kwargs = training_kwargs(
            task_priorities={params['tasks'][0]:'primary_task'},
            tasks=params['tasks'],
            class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
            early_stopping_patience=2,
            shared_encoder_n_layers=0,
            normalised_values=[40,0],
            frozen_layers=params['frozen_layers'],
            save_results=True)

        args = training_args(lr=params['lr'],epochs=8)
        model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)
        model = freeze_layers(model,frozen_layers=params['frozen_layers'],model_type=model_type)
        dev_set = 'dev' if params['tasks']!='ner' else 'validation'
        trainer = MultiTaskModelTrainer(
            model,
            args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict[dev_set],
        )
        trainer.kwargs_dict = trainer_kwargs
        trainer.test_dataset = dataset_dict['test']
        trainer.train()
        prev_task=params['tasks']

{'frozen_layers': None, 'lr': 1e-05, 'tasks': ['aes']}


/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

...

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "

KeyError: ignored

In [4]:
for pretrained_model_name in ['distilroberta-base']:
    model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
    model_n_layers = 6 if 'roberta' in pretrained_model_name else 12
    params_grid = sorted(create_params_grid({'tasks':[['ner']],
                                      'lr':[1e-5,4e-5,8e-5],
                                      'frozen_layers':[None,model_n_layers/2,model_n_layers]}
                                      )
                        , key=lambda k: k['tasks'])

    prev_task = None
    for i,params in enumerate(params_grid):
        print(params)
        if params['tasks']!=prev_task:
            dataset_obj,dataset_dict = get_dataset(params['tasks'],pretrained_model=pretrained_model_name,max_length=512)
            dataset_dict = dataset_dict.map(normalise_scores)

        trainer_kwargs,model_kwargs = training_kwargs(
            task_priorities={params['tasks'][0]:'primary_task'},
            tasks=params['tasks'],
            class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
            early_stopping_patience=2,
            shared_encoder_n_layers=0,
            normalised_values=[40,0],
            frozen_layers=params['frozen_layers'],
            save_results=True)

        args = training_args(lr=params['lr'],epochs=8)
        model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)
        model = freeze_layers(model,frozen_layers=params['frozen_layers'],model_type=model_type)
        dev_set = 'dev' if params['tasks'][0]!='ner' else 'validation'
        trainer = MultiTaskModelTrainer(
            model,
            args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict[dev_set],
        )
        trainer.kwargs_dict = trainer_kwargs
        trainer.test_dataset = dataset_dict['test']
        trainer.train()
        prev_task=params['tasks']

{'frozen_layers': None, 'lr': 1e-05, 'tasks': ['ner']}


/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

...

{'frozen_layers': 6, 'lr': 8e-05, 'tasks': ['ner']}


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    

Step,Training Loss


ner_weight_coef : 1.0

losses {'ner': 2.4163703629464814, 'total_loss': 2.4163703629464814}

ner               precision    recall  f1-score   support

           0  0.796664019 0.023457050 0.045572266     42759
           1  0.036072144 0.009771987 0.015378044      1842
           2  0.009018036 0.006885998 0.007809111      1307
           3  0.004975124 0.000745712 0.001297017      1341
           4  0.013828190 0.821571238 0.027198589       751
           5  0.044642857 0.002721829 0.005130836      1837
           6  0.010666667 0.015564202 0.012658228       257
           7  0.003132614 0.009761388 0.004743083       922
           8  0.035211268 0.043352601 0.038860104       346

    accuracy                      0.032728476     51362
   macro avg  0.106023435 0.103759112 0.017627475     51362
weighted avg  0.667023607 0.032728476 0.039714576     51362

ner_metrics {'accuracy_ner': 0.03272847630543982, 'f1_score_macro_ner': 0.017627475237132675, 'f1_score_weighted_ner': 0.039714575



Training completed. Do not forget to share your model on huggingface.co/models =)




In [6]:
for pretrained_model_name in ['bert-base-uncased']:
    model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
    model_n_layers = 6 if 'roberta' in pretrained_model_name else 12
    params_grid = sorted(create_params_grid({'tasks':[['ged']],
                                      'lr':[8e-5],
                                      'frozen_layers':[None]}
                                      )
                        , key=lambda k: k['tasks'])

    prev_task = None
    for i,params in enumerate(params_grid):
        print(params)
        if params['tasks']!=prev_task:
            dataset_obj,dataset_dict = get_dataset(params['tasks'],pretrained_model=pretrained_model_name,max_length=512)
            dataset_dict = dataset_dict.map(normalise_scores)

        trainer_kwargs,model_kwargs = training_kwargs(
            task_priorities={params['tasks'][0]:'primary_task'},
            tasks=params['tasks'],
            class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
            early_stopping_patience=2,
            shared_encoder_n_layers=0,
            normalised_values=[40,0],
            frozen_layers=params['frozen_layers'],
            save_results=True)

        args = training_args(lr=params['lr'],epochs=8)
        model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)
        model = freeze_layers(model,frozen_layers=params['frozen_layers'],model_type=model_type)
        dev_set = 'dev' if params['tasks']!='ner' else 'validation'
        trainer = MultiTaskModelTrainer(
            model,
            args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict[dev_set],
        )
        trainer.kwargs_dict = trainer_kwargs
        trainer.test_dataset = dataset_dict['test']
        trainer.train()
        prev_task=params['tasks']

{'frozen_layers': None, 'lr': 8e-05, 'tasks': ['ged']}


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

...

  0%|          | 0/79 [00:00<?, ?ex/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "

Step,Training Loss
500,0.539100


ged_weight_coef : 1.0

losses {'ged': 0.6767667664421929, 'total_loss': 0.6767667664421929}

ged               precision    recall  f1-score   support

           0       0.93      0.66      0.77     27374
           1       0.24      0.67      0.35      4283

    accuracy                           0.66     31657
   macro avg       0.58      0.67      0.56     31657
weighted avg       0.83      0.66      0.72     31657

ged_metrics {'accuracy_ged': 0.664845057965063, 'f1_score_macro_ged': 0.5621920191891366, 'f1_score_weighted_ged': 0.71682450130636, 'f_0_5_ged': 0.4899407067210474}

epochs since best performance 0

ged_weight_coef : 1.0

losses {'ged': 0.6901578373379178, 'total_loss': 0.6901578373379178}

ged               precision    recall  f1-score   support

           0       0.93      0.67      0.78     27374
           1       0.25      0.68      0.36      4283

    accuracy                           0.67     31657
   macro avg       0.59      0.68      0.57     31657
weighte

RuntimeError: ignored